In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from datasets import load_dataset
from perfcounters import PerfCounters

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = load_dataset("rotten_tomatoes", split="train")
test = load_dataset("rotten_tomatoes", split="test")
train_texts = [t['text'] for t in train]
test_texts = [t['text'] for t in test]
num_train_texts = len(train_texts)
num_test_texts = len(test_texts)
num_texts = num_train_texts + num_test_texts

In [4]:
import os
BATCH_SIZE = 256
STORE_DATA = False
APPROX = True

os.environ['BACKEND'] = 'onnx'  # or tf or comment for auto
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
from unisim import TextSim
if APPROX:
    tsim = TextSim(index_type="approx", batch_size=BATCH_SIZE)
else:
    tsim = TextSim(index_type="exact", batch_size=BATCH_SIZE)

tsim.info()

2023-10-28 21:19:12.384429: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-28 21:19:12.457513: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded backend
Using onnx with cpu
UniSim is storing a copy of the indexed data
If you are using large data corpus, consider disabling this behavior using store_data=False
Accelerator is not available, using cpu instead
[Embedder]
|-batch_size: 256
|-model_id: text/retsim/v1
|-embedding_size: 256
[Indexer]
|-index_type: approx
|-use_accelerator: False
|-store index data: True


/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'DmlExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [6]:
tsim.embed(['warm-up'])
tsim.reset_index()
tsim.add(test_texts[:20])
dups = tsim.search(test_texts[:20])
print(dups)

-[Results Statistics]-
Number of Results: 20
Total Global Matches: 20
Total Partial Matches: 20


In [7]:
tsim.reset_index()

cnts = PerfCounters()
cnts.start('total')

cnts.start('indexing-train')
tsim.add(train_texts)
cnts.stop('indexing-train')

cnts.start('indexing-test')
tsim.add(test_texts)
cnts.stop('indexing-test')

cnts.start('search-train')
train_dups = tsim.search(train_texts)
cnts.stop('search-train')

cnts.start('search-test')
test_dups = tsim.search(test_texts)
cnts.stop('search-test')

cnts.stop('total')

In [8]:
cnts.report()

-=[Timing counters]=-
+----------------+----------+
| name           |    value |
|----------------+----------|
| total          | 39.1854  |
| search-train   | 17.6514  |
| indexing-train | 17.4838  |
| search-test    |  2.08035 |
| indexing-test  |  1.96944 |
+----------------+----------+




In [9]:
# timings
counters = {}
for c in cnts.get_all()['Timing counters']:
    print(c)
    counters[c[0]] = c[1]

print("timing")
print(f"total: {round(counters['total'])} sec - {round(num_texts / counters['total'], 2)}  ex/s")
print(f"embedding_time: fixme")
print(f"indexing:fixme")

print("Train")
neardup_train_ratio = train_dups.total_global_matches / num_train_texts

iqps = num_train_texts / cnts.get('indexing-train')
sqps = num_test_texts / cnts.get('search-test')
print(f'indexing {round(iqps)} qps')
print(f'search {round(sqps)} qps')

['total', 39.18539905548096]
['search-train', 17.651363134384155]
['indexing-train', 17.483804941177368]
['search-test', 2.0803515911102295]
['indexing-test', 1.9694361686706543]
timing
total: 39 sec - 244.89  ex/s
embedding_time: fixme
indexing:fixme
Train
indexing 488 qps
search 512 qps


In [10]:
t1 = train_texts[2062]
t2 = "lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness ."
tsim.similarity(t1, t2)

0.5280479192733765

In [11]:
# import numpy as np

# [0.1, 0.3, 0.4,0.5]/np.linalg.norm([0.1, 0.3, 0.4,0.5],axis=0, keepdims=True)

In [12]:
# import tensorflow as tf
# tf.math.l2_normalize([0.1, 0.3, 0.4,0.5], axis=0)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("/workspace/RetSim/models/retsim_infinity/retsim_infinite_model_v1")
e1 = model.predict(tf.constant([t1]))
e2 = model.predict(tf.constant([t2]))

# with tf.device()
tf.matmul(e1["global_embedding"], e2["global_embedding"], transpose_b=True)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


1/1 [==============================] - 0s 61ms/step


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.52804935]], dtype=float32)>

In [13]:
# set True to store data to show the cotent
for d in test_dups.results[0:]:
    if d.num_global_matches:
        tsim.visualize(d)
        break

Query 0: "lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness ."
  idx  is_global      global_sim  is_partial      partial_sim  content
-----  -----------  ------------  ------------  -------------  --------------------------------
 8530  True                 1     True                   1     lovingly photographed in the man
 8979  False                0.57  False                  0.57  waydowntown manages to nail the
 8960  False                0.56  False                  0.56  manages to delight without much
  909  False                0.56  False                  0.56  culkin , who's in virtually ever
 8209  False                0.54  False                  0.54  unfortunately , one hour photo l


In [ ]:
wiki_time = (3_000_000 / iqps)
wiki_hour = wiki_time / 3600
print(f"{round(wiki_time)} sec, {round(wiki_hour, 1)}h")

6542 sec, 1.8h
